# Mean-Reverting Vector Portfolios
This notebook implements the algorithms from Aspremont et al. (MeanRevVec) to find mean-reverting portfolios over large datasets. It uses Finnhub to fetch market data (if API key is provided) and provides Python implementations of PCA-based discovery, Engle-Granger cointegration tests, Ornstein-Uhlenbeck parameter estimation, and a simple mean-reversion trading simulation.

We include equations and explanations inline.

## 1. Mathematical Background
Let P_t be the vector of asset prices. We look for a weight vector w such that the portfolio (or spread) s_t = w^	op og P_t is mean-reverting. The paper formulates several objectives; a common approach is to search for low-variance directions (PCA) or maximize a mean-reversion criterion.

Ornstein-Uhlenbeck process (continuous-time) used as model for the spread:
$$athrm{d}s_t = 	heta(u - s_t)athrm{d}t + igma athrm{d}W_t$$
Discretization and parameter estimation are described below.

In [ ]:
# Imports
import pandas as pd
import numpy as np
from python.meanrev import fetch_price_series, compute_log_returns, pca_portfolios, engle_granger_test, estimate_ou_params, backtest_portfolio

# Display settings
pd.options.display.max_columns = 50
pd.options.display.width = 120

In [ ]:
# Parameters
symbols = ['AAPL','MSFT','GOOGL','AMZN','META']
start = '2024-01-01'
end = '2025-11-01'
freq = '1D'

# Fetch data (uses Finnhub if available, otherwise synthetic)
prices = pd.DataFrame({s: fetch_price_series(None, s, start, end, freq=freq) for s in symbols})
prices.tail()

## PCA portfolios
We compute PCA on log-returns and inspect principal components as candidate weight vectors.

In [ ]:
returns = compute_log_returns(prices)
comps, pca = pca_portfolios(returns, n_components=3)
pca.explained_variance_ratio_

In [ ]:
# Evaluate OU params for PC1 portfolio
w = comps[0]
port = (prices * w).sum(axis=1)
ou = estimate_ou_params(np.log(port))
ou

In [ ]:
# Backtest
res = backtest_portfolio(w, prices, entry_z=1.5, exit_z=0.5)
res['sharpe'], res['equity'].tail()

## Next steps
- Implement the mean-reversion maximization objective from the paper (optimization over w).
- Port heavy linear algebra and optimization routines to Rust and expose them via the existing Rust connector.
- Add parallel data ingestion for large symbol universes.